<a href="https://colab.research.google.com/github/dilanbakr/artificalQuestionPredict/blob/main/sorucevap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from keras.layers import Input, Dense, Embedding, Concatenate, Flatten, Dropout
from keras.models import Model, Sequential
from keras import optimizers
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from scipy import spatial

In [ ]:
ratings = pd.read_csv("sorucevapDataset.txt")

In [ ]:
ratings.head()

,userId,soruId,rating,answerGenres,genres
0,1,1,1,Y,kesirler
1,1,2,1,D,integral
2,1,3,1,Y,dil bilgisi
3,1,4,1,B,turev
4,1,5,2,Y,kesirler


In [ ]:
ansGenres= ratings.groupby(['userId','answerGenres']).count()
ansGenres

soruId  rating  genres
userId answerGenres                        
1      B                  5       5       5
       D                  7       7       7
       Y                  8       8       8
2      B                  2       2       2
       D                  8       8       8
       Y                 10      10      10
3      D                 16      16      16
       Y                  4       4       4
4      B                  5       5       5
       D                  2       2       2
       Y                 13      13      13

In [ ]:
 def create_dataset(ratings):
    ratings.groupby('userId')['rating'].count()
    unique_users = ratings.userId.unique()
    user_to_index = {old: new for new, old in enumerate(unique_users)}
    new_users = ratings.userId.map(user_to_index)
    unique_soru = ratings.soruId.unique()
    soru_to_index = {old: new for new, old in enumerate(unique_soru)}
    new_soru = ratings.soruId.map(soru_to_index)
    n_users = unique_users.shape[0]
    n_soru = unique_soru.shape[0]
    X = pd.DataFrame({'user_id': new_users, 'soru_id': new_soru})
    y = ratings['rating'].astype(np.float32)
    return (n_users, n_soru), (X, y), (user_to_index, soru_to_index)

In [ ]:
(n, m), (X, y), (user_to_index,soru_to_index) = create_dataset(ratings)
print(f'Embeddings: {n} users, {m} movies')
print(f'Dataset shape: {X.shape}')
print(f'Target shape: {y.shape}')

Embeddings: 4 users, 20 movies
Dataset shape: (80, 2)
Target shape: (80,)


In [ ]:
ratings["user_id_index"] = ratings["userId"].apply(lambda x: user_to_index[x])

In [ ]:
ratings["soru_id_index"] = ratings["soruId"].apply(lambda x: soru_to_index[x])

In [ ]:
#öğrenci seviyesini belirliyorum
ansGenres= ratings.groupby(['userId','answerGenres']).count()
userLevelarray = []
unique_users = ratings.userId.unique()
countUniqeSoru = ratings.soruId.unique().size
countTrueSoru = ratings[ratings['answerGenres'] =='D']
for i in unique_users:
  userLevel = (countTrueSoru[countTrueSoru['userId'] == i]['soruId'].count()/countUniqeSoru)*10
  userLevelarray.append(int(userLevel))
  #print(userLevel)
userLevelarray
ratings["studentlevel"] = ratings["user_id_index"].apply(lambda x: userLevelarray[x])
ratings.head(21)  

,userId,soruId,rating,answerGenres,genres,user_id_index,soru_id_index,studentlevel
0,1,1,1,Y,kesirler,0,0,3
1,1,2,1,D,integral,0,1,3
2,1,3,1,Y,dil bilgisi,0,2,3
3,1,4,1,B,turev,0,3,3
4,1,5,2,Y,kesirler,0,4,3
5,1,6,2,D,integral,0,5,3
6,1,7,2,D,dil bilgisi,0,6,3
7,1,8,2,D,turev,0,7,3
8,1,9,3,D,kesirler,0,8,3
9,1,10,3,Y,integral,0,9,3


In [ ]:
sl = ratings["studentlevel"].unique().shape[0]
gn = ratings["genres"].unique().shape[0]
ratings["studentlevel"].unique()

array([3, 4, 8, 1])

In [ ]:
sl

4

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
y_normalized = scaler.fit_transform(y.to_numpy().reshape(-1, 1))
y_normalized = y_normalized.ravel()

In [ ]:
y_normalized

array([0.  , 0.  , 0.  , 0.  , 0.25, 0.25, 0.25, 0.25, 0.5 , 0.5 , 0.5 ,
       0.5 , 0.75, 0.75, 0.75, 0.75, 1.  , 1.  , 1.  , 1.  , 0.  , 0.  ,
       0.  , 0.  , 0.25, 0.25, 0.25, 0.25, 0.5 , 0.5 , 0.5 , 0.5 , 0.75,
       0.75, 0.75, 0.75, 1.  , 1.  , 1.  , 1.  , 0.  , 0.  , 0.  , 0.  ,
       0.25, 0.25, 0.25, 0.25, 0.5 , 0.5 , 0.5 , 0.5 , 0.75, 0.75, 0.75,
       0.75, 1.  , 1.  , 1.  , 1.  , 0.  , 0.  , 0.  , 0.  , 0.25, 0.25,
       0.25, 0.25, 0.5 , 0.5 , 0.5 , 0.5 , 0.75, 0.75, 0.75, 0.75, 1.  ,
       1.  , 1.  , 1.  ], dtype=float32)

In [ ]:
input_user = Input(shape=(1,), dtype='int32', name='user_input')
input_soru = Input(shape=(1,), dtype='int32', name='soru_input')
input_studentlevel = Input(shape=(1,), dtype='int32', name='student_input')

x_1 = Embedding(output_dim=50, input_dim=n, input_length=1, name="user_embeddings")(input_user)
x_2 = Embedding(output_dim=50, input_dim=m, input_length=1, name="soru_embeddings")(input_soru)
x_3 = Embedding(output_dim=50, input_dim=sl+10, input_length=1, name="studentlevel_embeddings")(input_studentlevel)


x = Concatenate()([x_1, x_2, x_3])
x1_f = Flatten()(x)
x1 = Dropout(0.05)(x1_f)
x2_f = Dense(300, activation='relu')(x1)
x2 = Dropout(0.5)(x2_f)
x3_f = Dense(300, activation='relu')(x2)
x3 = Dropout(0.5)(x3_f)
x4 = Dense(300, activation='relu')(x3)
x4_f = Dropout(0.25)(x4)
pred = Dense(1, activation='linear')(x4_f)

model = Model(inputs=[input_user, input_soru, input_studentlevel], outputs=pred)
model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mse'])

In [ ]:
model.summary()

Model: "model_23"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_input (InputLayer)         (None, 1)            0                                            
__________________________________________________________________________________________________
soru_input (InputLayer)         (None, 1)            0                                            
__________________________________________________________________________________________________
student_input (InputLayer)      (None, 1)            0                                            
__________________________________________________________________________________________________
user_embeddings (Embedding)     (None, 1, 50)        200         user_input[0][0]                 
___________________________________________________________________________________________

In [ ]:
model.fit([X.user_id.to_numpy(), X.soru_id.to_numpy(),ratings['studentlevel'].to_numpy()], y_normalized,
          epochs=10,
          batch_size=256,
          validation_split=0.25
          )

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 60 samples, validate on 20 samples
Epoch 1/10
60/60 [==============================] - 0s 4ms/step - loss: 0.3894 - mse: 0.3894 - val_loss: 0.3283 - val_mse: 0.3283
Epoch 2/10
60/60 [==============================] - 0s 163us/step - loss: 0.3271 - mse: 0.3271 - val_loss: 0.2853 - val_mse: 0.2853
Epoch 3/10
60/60 [==============================] - 0s 180us/step - loss: 0.2776 - mse: 0.2776 - val_loss: 0.2416 - val_mse: 0.2416
Epoch 4/10
60/60 [==============================] - 0s 189us/step - loss: 0.2223 - mse: 0.2223 - val_loss: 0.1994 - val_mse: 0.1994
Epoch 5/10
60/60 [==============================] - 0s 185us/step - loss: 0.1693 - mse: 0.1693 - val_loss: 0.1611 - val_mse: 0.1611
Epoch 6/10
60/60 [==============================] - 0s 168us/step - loss: 0.1369 - mse: 0.1369 - val_loss: 0.1314 - val_mse: 0.1314
Epoch 7/10
60/60 [==============================] - 0s 183us/step - loss: 0.1260 - mse: 0.1260 - val_loss: 0.1157 - val_mse: 0.1157
Epoch 8/10
60/60 [================

In [ ]:
ratings.head()

,userId,soruId,rating,answerGenres,genres,user_id_index,soru_id_index,studentlevel
0,1,1,1,Y,kesirler,0,0,3
1,1,2,1,D,integral,0,1,3
2,1,3,1,Y,dil bilgisi,0,2,3
3,1,4,1,B,turev,0,3,3
4,1,5,2,Y,kesirler,0,4,3


In [ ]:
ratings.loc[ratings["userId"] == 2, :].head(8)

,userId,soruId,rating,answerGenres,genres,user_id_index,soru_id_index,studentlevel
20,2,1,1,D,kesirler,1,0,4
21,2,2,1,D,integral,1,1,4
22,2,3,1,Y,dil bilgisi,1,2,4
23,2,4,1,B,turev,1,3,4
24,2,5,2,Y,kesirler,1,4,4
25,2,6,2,Y,integral,1,5,4
26,2,7,2,D,dil bilgisi,1,6,4
27,2,8,2,D,turev,1,7,4


In [ ]:
model.layers

In [ ]:
user_embed = pd.DataFrame(model.layers[3].get_weights()[0])
soru_embed = pd.DataFrame(model.layers[4].get_weights()[0])
level_embed = pd.DataFrame(model.layers[5].get_weights()[0])

In [ ]:
user_embed

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
0,0.017241,0.055932,0.038490,0.028566,0.048685,-0.034568,0.013992,-0.041647,0.044890,-0.042000,-0.005260,0.042108,-0.026686,0.045796,0.000309,0.004880,-0.030028,-0.008043,-0.048597,-0.048841,-0.027477,-0.046103,-0.018594,0.020875,0.011929,0.042080,-0.042764,0.018054,-0.044973,0.031560,0.004210,0.007165,0.014731,-0.047644,-0.037464,-0.024521,0.011158,-0.001593,-0.033306,0.046601,0.015313,-0.054717,-0.034426,-0.015404,0.001997,0.010744,0.005132,-0.039960,0.031903,-0.014014
1,-0.017003,-0.040365,0.002601,0.011140,0.049274,-0.036382,-0.035505,0.012902,0.012590,0.047380,-0.042035,-0.005781,-0.040193,0.025687,0.004100,-0.022758,-0.031483,0.002298,-0.026372,-0.015176,0.026914,-0.015141,-0.021999,0.007901,0.011267,0.018118,-0.004774,-0.022655,0.039012,-0.024742,-0.054991,0.015223,0.006125,0.042326,-0.000026,-0.031992,0.047720,0.026732,0.025831,0.011991,-0.004249,0.022741,0.007694,0.040640,-0.054487,-0.048106,0.010314,-0.043404,0.000292,-0.005004
2,0.002857,-0.035648,-0.045415,-0.036414,-0.004382,0.003347,-0.018356,-0.004752,0.048096,-0.022060,-0.032671,-0.000530,-0.021325,-0.042603,0.028928,0.024260,0.020297,-0.038974,0.054462,-0.038328,0.028914,0.000685,0.000245,0.020903,0.033974,0.049041,0.026340,-0.005362,0.000736,0.005929,0.004046,-0.050533,-0.014348,0.020084,0.031170,0.027488,0.035347,0.019498,-0.032295,0.051772,0.039048,-0.040432,-0.031150,0.036384,0.037819,0.027593,0.031928,-0.016381,0.018638,-0.048113
3,-0.014778,-0.032282,0.011576,-0.026422,0.049875,-0.000166,-0.038184,-0.018639,-0.012646,-0.023925,-0.007892,-0.047706,0.019332,0.010906,-0.020673,-0.036705,-0.018073,0.008159,-0.026115,0.018715,0.019683,0.020858,-0.038903,-0.023967,0.045761,0.030022,-0.017873,0.030384,0.035784,0.036560,0.029707,0.041595,0.034037,0.024203,0.040937,-0.021735,-0.029910,0.002253,0.014783,0.008018,-0.046544,-0.008072,0.043315,0.037699,0.046587,0.034282,0.015558,0.036094,0.038965,0.013259


In [ ]:
soru_embed

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
0,-0.032539,0.046921,-0.023891,0.009519,-0.018437,0.034235,0.020067,0.022758,0.029107,-0.004683,0.018762,0.015593,0.027857,-0.005137,0.014473,-0.044742,-0.025685,0.040728,-0.050767,-0.018922,-0.002798,0.018096,0.004130,-0.028461,-0.009340,-0.044456,-0.024865,0.003185,-0.027386,0.002937,0.036331,-0.002354,0.014506,0.017938,-0.012450,0.027721,-0.045467,0.047131,-0.016430,-0.005399,-0.021895,-0.006181,0.033841,0.022069,0.017722,-0.014724,-0.029625,0.021406,0.001219,-0.041436
1,0.006556,0.014148,0.042235,-0.011806,0.024281,-0.046687,-0.037649,0.021308,0.007421,-0.004446,0.027688,0.027934,0.041222,-0.019840,0.025985,-0.007791,-0.012530,0.039011,0.035702,0.020770,0.028475,0.028999,-0.045796,0.043086,0.006765,0.020276,0.003359,-0.007327,-0.001154,0.004429,0.034048,-0.040430,0.037316,0.025766,0.027516,0.022573,0.013678,0.026856,-0.041249,0.002036,0.045593,-0.007556,0.035876,0.015713,-0.032009,-0.041615,0.030166,-0.008946,-0.004254,-0.039827
2,0.013041,-0.004818,0.044497,-0.047216,0.001919,0.027784,0.036338,-0.037963,-0.019517,-0.029302,-0.013674,-0.033945,-0.006270,-0.030159,-0.017063,-0.015943,-0.047903,0.018134,-0.034008,0.024891,0.038348,0.019454,0.019104,-0.039556,0.038409,0.034550,-0.007336,-0.048010,0.009176,0.002067,0.004712,-0.020376,-0.018435,0.034777,0.030879,-0.024020,-0.004285,-0.018890,0.040855,0.029805,-0.000317,-0.002430,0.039457,-0.046929,-0.019864,-0.017526,0.003892,0.039922,0.044774,0.016475
3,-0.002530,0.038547,-0.027659,-0.036388,0.021551,-0.005211,0.029428,-0.036028,-0.046612,0.014738,-0.004034,0.028612,0.042545,0.032559,-0.043314,-0.026139,0.038319,0.016000,0.031908,-0.010301,0.039967,0.000554,-0.043812,-0.003058,0.020990,0.037347,-0.008800,-0.012450,0.004089,0.006737,-0.014195,-0.011151,0.035973,0.005283,-0.014619,0.029764,0.031612,0.025344,0.001970,-0.041015,0.009339,0.034422,-0.035743,0.015147,0.003434,0.003030,-0.027755,0.028699,0.018385,0.015398
4,-0.030530,-0.005252,-0.000243,-0.046605,0.038285,-0.022978,0.004295,-0.006087,0.005967,-0.049106,-0.007918,0.004547,0.040486,-0.005408,0.029663,-0.011904,-0.015676,-0.018055,-0.033571,0.014546,-0.050967,0.034072,0.040711,-0.014353,0.035331,-0.014453,0.029563,-0.003245,-0.013600,-0.033165,-0.027737,0.028993,-0.049772,0.035420,-0.038314,0.019273,0.023212,0.024814,0.022766,0.045520,-0.043248,0.001476,-0.019274,-0.013794,-0.039984,0.013796,0.000644,-0.024218,-0.025101,0.013188
5,-0.026592,0.035772,0.013703,-0.044777,-0.034142,0.011645,0.019835,-0.031166,-0.041896,-0.009372,-0.010485,-0.027064,-0.005443,0.022066,0.013411,0.046083,0.032520,-0.014918,0.016185,-0.018557,0.033943,0.027785,0.006562,0.028535,0.001314,0.018230,-0.014098,0.020405,-0.000100,0.027708,-0.021579,0.042302,-0.017030,-0.044412,0.017498,0.052358,0.043477,-0.017669,0.040693,0.035830,-0.012084,-0.024062,0.026494,0.045274,0.036870,-0.020064,-0.038594,0.043271,0.019631,0.005602
6,-0.036510,0.036575,-0.029906,0.009007,-0.002403,-0.004359,0.006168,-0.043598,-0.038252,-0.012506,-0.046591,-0.022958,0.045255,0.036861,-0.042784,0.000921,0.038125,-0.043393,-0.034794,0.013054,0.026022,0.016895,-0.051831,-0.010381,0.012035,0.021347,0.038139,0.040998,0.013485,-0.031549,0.024618,0.040345,-0.003558,-0.035653,-0.006175,0.035081,-0.019497,0.008058,0.039251,0.007927,0.030490,0.014100,0.004762,0.026722,-0.043566,-0.035612,0.045984,-0.022571,0.003862,-0.016744
7,-0.009538,-0.033011,-0.033521,-0.015283,0.018022,-0.035642,-0.047445,0.005364,-0.025923,0.010168,0.039713,0.019984,0.019138,-0.047439,-0.006354,0.027565,0.031717,0.028814,-0.009288,-0.041937,0.042801,0.040705,-0.022120,0.020788,-0.022962,0.023430,-0.020391,-0.039462,0.021957,0.020057,0.010851,-0.035465,0.027376,0.004002,-0.027575,-0.032224,-0.037029,0.032225,0.034710,0.040651,-0.027922,-0.023527,0.007258,-0.040591,-0.046140,0.008006,0.017724,0.018574,-0.031781,-0.032828
8,-0.026093,-0.045074,0.028910,0.044391,0.017673,-0.02481

In [ ]:
class color:
    PURPLE = '\033[95m'
    CYAN = '\033[96m'
    DARKCYAN = '\033[36m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'

In [ ]:
def findMostSimilarSoru(ratings, soru_embed, soru_id):
    
    selectedSoru_index = ratings.loc[ratings["soruId"] == soru_id, "soru_id_index"].unique()[0]
    
    selected_embedding = soru_embed.loc[selectedSoru_index, :].to_numpy()
    soru_embed = soru_embed.to_numpy()
    
    distance = np.zeros(soru_embed.shape[0])
    for i, soru in enumerate(soru_embed):
        distance[i] = 1 - spatial.distance.cosine(selected_embedding, soru)
    most_similars = np.argsort(distance)[-2:-11:-1]
    
    print("\n")
    print("Tasviye edilen Sorular")
    print("=" * 20)
    for sim in most_similars:
        rec_soru_genres = ratings.loc[ratings["soru_id_index"] == sim , "genres"].unique()[0]
        print(color.BOLD + color.BLUE + "{0} nolu soru ".format( sim))
    
    return None

In [ ]:
findMostSimilarSoru(ratings, soru_embed, 2)



Tasviye edilen Sorular
7 nolu soru 
8 nolu soru 
3 nolu soru 
0 nolu soru 
9 nolu soru 
10 nolu soru 
17 nolu soru 
6 nolu soru 
2 nolu soru 
